# Tutorial 3: Other Methods
In this tutorial, we'll describe how to incorporate other Lipschitz estimation techniques into our codebase. For each method, we try to use the official codebase attached to the papers. We simply built an interface on top of each of these to be more amenable to our system. The methods we consider are:
* **CLEVER**: Uses randomly sampled points and extremal value theory to generate an heuristic Lipschitz estimate([github](https://github.com/IBM/CLEVER-Robustness-Score)). 
* **FastLip**: Uses the hyperbox and boolean hyperbox abstract domains to efficiently generate an upper bound to the Lipschitz constant ([github](https://github.com/huanzhang12/CertifiedReLURobustness)).
* **LipLP**: The naive linear programming relaxation to LipMIP
* **LipSDP**: Uses incremental quadratic constraints and semidefinite programming to generate a global upper bound of Lipschitz constants in the $\ell_2$ setting ([github](https://github.com/arobey1/LipSDP)).
* **SeqLip**: Frames Lipschitz estimation as a combinatorial optimization problem and uses greedy methods to generate a heuristic Lipschitz estimate ([github](https://github.com/avirmaux/lipEstimation)).
* **RandomLB**: Randomly samples points and takes their maximal gradient norm. This is like CLEVER, but doesn't use the extremal value theory step, and thereby provides a certifiable lower bound.
* **NaiveUB**: Multiplies the operator norm of each component of a ReLU network together to yield an extremely loose upper bound.
* **ZLip**: Uses zonotope abstract interpretations to overapproximate output range

Note that LipSDP uses the Mosek plugin for matlab to solve SDP's. Follow the instructions in their github to install these dependencies.

In [ ]:
# Imports
# import matlab.engine

import sys
sys.path.append('..')
import torch 
from pprint import pprint 

import utilities as utils
from relu_nets import ReLUNet
from hyperbox import Hyperbox 
from lipMIP import LipMIP
from other_methods import CLEVER, FastLip2, LipLP, LipSDP, NaiveUB, RandomLB, SeqLip, ZLip
import experiment as exp 

# 1: Individual Methods
The interface to run each method is identical, and all inherit a generic `OtherResult` class (except for LipMIP). We demonstrate how to run each method here.

Many methods have variants and hyperparameters that can be tuned. We incorporate these as kwargs and can tune them in our repository, but leave them mostly as default from their original codebases.

In [ ]:
# Basic network example 
test_network = ReLUNet([2, 8, 16, 2])
test_domain = Hyperbox.build_unit_hypercube(2)
primal_norm = 'linf'
c_vector = torch.Tensor([1.0, -1.0])

In [ ]:
# CLEVER (the first one alphabetically, so we'll use this as an example)
test_clever = CLEVER(test_network, c_vector, test_domain, primal_norm) # builds a CLEVER instance
clever_out = test_clever.compute() # executes CLEVER 
# Let's examine the output of CLEVER for a moment
print('Lipschitz estimate (as a float) is:', clever_out)

# We also store these attributes in the test_clever object (of class CLEVER)
print('test_clever holds the value (%f), and compute_time (%f s)' % (test_clever.value, test_clever.compute_time))


In [ ]:
# Evaluating all the methods (using the same interface)
for other_method in [CLEVER, 
                     FastLip2,
                     LipLP,
                     NaiveUB, RandomLB, SeqLip, ZLip]:
    test_object = other_method(test_network, c_vector, domain=test_domain, primal_norm=primal_norm)
    test_object.compute()
    print(other_method.__name__ + ' ran in %.02f seconds and has value %.04f' % 
          (test_object.compute_time, test_object.value))

# CAVEAT! Some methods, such as LipSDP output an l2-lipschitz value, which needs to be scaled according to dimension

# 2: The `Experiment` class
As a convenient and flexible shorthand to evaluate lipschitz constants of various networks under various settings, we built the `Experiment` class which is very handy for performing common operations.

In [ ]:
eval_class = [#CLEVER, 
              FastLip2, 
              LipLP, 
              NaiveUB, 
              RandomLB, 
              SeqLip, 
              LipMIP, 
              ZLip]

In [ ]:
# Use case 1: evaluating local lipschitz constants for a fixed network, across many small random domains

# --- build experiment object 
basic_exp = exp.Experiment(eval_class, network=test_network, c_vector=c_vector, primal_norm=primal_norm)

# --- run all methods across 10 random hyperboxes, centered in [0,1]^2, with fixed radius 0.2
sample_domain = Hyperbox.build_unit_hypercube(2)
sample_factory = utils.Factory(Hyperbox.build_linf_ball, radius=0.2)
random_out = basic_exp.do_random_evals(3, sample_domain, sample_factory) # This should take about a minute

In [ ]:
'''Examining random_out...
random_out is a ResultList object, which is a list-like wrapper for individual Result Objects
We'll first interact with a Result object which considers local lipschitz estimation for only one of the 3 
random points we evaluated above. ResultList allows us to collect average and standard deviations for values
and compute times
'''
result_0 = random_out.results[0] # first result 
result_0

In [ ]:
# Just the values:
pprint(result_0.values())

In [ ]:
# Just the compute times 
pprint(result_0.compute_times())

In [ ]:
# The entire objects
pprint(result_0.objects())

In [ ]:
# To operate on a result_list, you can access mean and standard deviations directly:
print('values:')
pprint(random_out.average_stdevs('value'))
print('\ntimes:')
pprint(random_out.average_stdevs('time'))

# where each triple is of the form (average, stdev, n)

In [ ]:
# Use case 2: evaluating local lipschitz constants across [0,1]^d
hcube_result = basic_exp.do_unit_hypercube_eval()
hcube_result.values()

In [ ]:
# Use case 3: evaluating local lipschitz constants across [-r, +r]^d where r is a parameter taken to be large
large_r_result = basic_exp.do_large_radius_eval(1000.0)
large_r_result.values()